In [ ]:
!pip install datasets
!pip install pandas

from collections import defaultdict
from datasets import load_dataset
import pandas as pd
import json



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from openai import OpenAI
client = OpenAI()

import openai

openai.api_key = ""

client.files.create(
  file=open("synthetic_qa_pairs_finetune_gpt3.5.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-MNOOpCzRXsKM62tzknqCXeBI', bytes=35974, created_at=1728021223, filename='synthetic_qa_pairs_finetune_gpt3.5.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
from collections import defaultdict
with open("synthetic_qa_pairs_finetune_gpt3.5.jsonl", 'r', encoding='utf-8') as f:
    file = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(file))
print("First example:")
for message in file[0]["messages"]:
    print(message)

Num examples: 97
First example:
{'role': 'system', 'content': 'You are a helpful assistant.'}
{'role': 'user', 'content': 'How do I unsubscribe from the monthly plan?'}
{'role': 'assistant', 'content': "To unsubscribe from the monthly plan, go to the subscription management dropdown in your dashboard, select 'Manage Subscription,' choose the 'Cancel Subscription' option, and confirm your decision. You will retain access to premium features until the current billing period ends."}


In [ ]:
format_errors = defaultdict(int)

for ex in file:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name", "function_call") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        function_call = message.get("function_call", None)

        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [ ]:
!pip install tiktoken
import tiktoken # for token counting
import numpy as np

encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in file:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 65536 for l in convo_lens)


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 48, 93
mean / median: 68.58762886597938, 68.0
p5 / p95: 58.6, 79.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 20, 58
mean / median: 37.1958762886598, 37.0
p5 / p95: 27.6, 47.0


In [ ]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(file)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~6653 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~19959 tokens


<h1>Finetuning the model</h1>

In [ ]:
from openai import OpenAI
client = OpenAI()


In [ ]:
response = client.files.create(
    file=open("synthetic_qa_pairs_finetune_gpt3.5.jsonl", "rb"),
    purpose="fine-tune"
)
print(response)

FileObject(id='file-IHFYm01BEBRTA9bQgaukDMK2', bytes=35974, created_at=1728021232, filename='synthetic_qa_pairs_finetune_gpt3.5.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)


In [ ]:
file_id = response.id
print(file_id)

file-IHFYm01BEBRTA9bQgaukDMK2


In [ ]:
response = client.fine_tuning.jobs.create(
    training_file=file_id,
    model="gpt-4o-mini-2024-07-18",
    hyperparameters={"n_epochs": 2}
    #validation_file
)

print("Fine-tune job is started")
ft_job_id = response.id

Fine-tune job is started


In [ ]:
response = client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_job_id, limit=10)
response


SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-2jQi1tuvmoMzSF1m5w0YKf9D', created_at=1728021233, level='info', message='Validating training file: file-IHFYm01BEBRTA9bQgaukDMK2', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-JuHJdJgRAdRoz5yeNiZE0eX2', created_at=1728021233, level='info', message='Created fine-tuning job: ftjob-fVGoMJ1bNB8YWJdHNFzqpX3F', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [ ]:
response = client.fine_tuning.jobs.retrieve(ft_job_id)
print(response)

FineTuningJob(id='ftjob-fVGoMJ1bNB8YWJdHNFzqpX3F', created_at=1728021233, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=2, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-RE1PxkWIlBhkHiAvwXqkJEQi', result_files=[], seed=1786541492, status='validating_files', trained_tokens=None, training_file='file-IHFYm01BEBRTA9bQgaukDMK2', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)


In [ ]:
result_files = response.result_files
result_files

[]

In [ ]:
fine_tuned_model = response.fine_tuned_model
fine_tuned_model

NameError: name 'response' is not defined

In [ ]:
!pip install openai==0.28.0
import openai

openai.api_key = ""

user_content = "What is the purpose of NDIS? And how to login into the system?"

# Create a chat completion using the fine-tuned model
completion = openai.ChatCompletion.create(
    model="ft:gpt-4o-mini-2024-07-18:kdyslab-ai::AEVYHYT8",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_content}
    ]

)

# Print the response content
print(completion.choices[0].message['content'])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 941.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.9/447.9 kB 14.2 MB/s eta 0:00:00
The National Disability Insurance System (NDIS) is designed to manage and facilitate the delivery of services and supports for individuals with disabilities. It serves as a platform for registering users, accessing resources, tracking service delivery, and maintaining communication with service providers.

To log in, navigate to the NDIS portal, enter your registered email address and password, and click the 'Login' button. If you encounter issues, use the 'Forgot Password' link to reset your password.


In [ ]:
pip install --upgrade openai


In [ ]:
import openai
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


# Validation data format: a list of dictionaries with 'input' and 'expected_output'
validation_data = [
    {"input": "What is the purpose of NDIS?", "expected_output": "The NDIS is designed to provide support to Australians with a disability."},
    {"input": "How do I log in to the system?", "expected_output": "To log in, visit the login page and enter your credentials."},
    # Add more validation data as needed
]

# Replace with your fine-tuned model name or ID (e.g., "chatgpt4omini")
fine_tuned_model = ""

# Store the predicted and expected outputs
predictions = []
expected_outputs = []

# Loop over each example in the validation data
for item in validation_data:
    user_input = item['input']
    expected_output = item['expected_output']

    # Generate a response using the fine-tuned chat model
    response = openai.ChatCompletion.create(
        model=fine_tuned_model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": user_input}
        ],
        max_tokens=100,
        temperature=0.5,
    )

    # Extract the content of the assistant's response
    predicted_output = response.choices[0].message.content.strip()

    # Append the predictions and expected outputs
    predictions.append(predicted_output)
    expected_outputs.append(expected_output)

# Calculate accuracy
accuracy = accuracy_score(expected_outputs, predictions)

# Calculate precision, recall, and F1-score
precision, recall, f1, _ = precision_recall_fscore_support(expected_outputs, predictions, average='weighted')

# Print the results
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
import openai
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import re

# Normalize text function
def normalize_text(text):
    text = re.sub(r'[^\w\s]', '', text).lower().strip()
    return text


# Validation data format: a list of dictionaries with 'input' and 'expected_output'
validation_data = [
    {"input": "What is the purpose of NDIS?", "expected_output": "The NDIS is designed to provide support to Australians with a disability."},
    {"input": "How do I log in to the system?", "expected_output": "To log in, visit the login page and enter your credentials."},
    # Add more validation data as needed
]

fine_tuned_model = ""

predictions = []
expected_outputs = []

# Loop over each example in the validation data
for item in validation_data:
    user_input = item['input']
    expected_output = item['expected_output']

    response = openai.ChatCompletion.create(
        model=fine_tuned_model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": user_input}
        ],
        max_tokens=100,
        temperature=0.5,
    )

    predicted_output = response.choices[0].message.content.strip()

    print(f"User Input: {user_input}")
    print(f"Expected Output: {expected_output}")
    print(f"Predicted Output: {predicted_output}\n")

    predictions.append(predicted_output)
    expected_outputs.append(expected_output)

# Normalize text for evaluation
predictions_normalized = [normalize_text(pred) for pred in predictions]
expected_outputs_normalized = [normalize_text(exp) for exp in expected_outputs]

# Calculate accuracy
accuracy = accuracy_score(expected_outputs_normalized, predictions_normalized)

# Calculate precision, recall, and F1-score
precision, recall, f1, _ = precision_recall_fscore_support(
    expected_outputs_normalized, predictions_normalized, average='weighted', zero_division=1
)

# Print the results
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
